<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### 1068. Product Sales Analysis I

```mysql
SELECT p.product_name, s.year, s.price
FROM sales s
JOIN product p
ON s.product_id=p.product_id
```

### 1069. Product Sales Analysis II

```mysql
SELECT product_id, sum(quantity) AS total_quantity
FROM sales
GROUP BY product_id
```

### 1070. Product Sales Analysis III

```mysql
-- MYSQL
SELECT s.product_id, s.year AS first_year, s.quantity, s.price
FROM Sales s
JOIN (
    SELECT product_id, MIN(year) AS y1
    FROM Sales
    GROUP BY product_id) t
ON (s.product_id=t.product_id AND s.year=t.y1)

/*    SELECT s1.product_id, s1.year as first_year, s1.quantity, s1.price
    FROM sales s1
    JOIN sales s2
    ON s1.product_id=s2.product_id AND s1.year>=s2.year
    GROUP BY s1.product_id, s1.year
    HAVING COUNT(s1.year)=1
*/

-- SQL server
SELECT product_id, year as first_year, quantity, price
FROM (
    SELECT product_id, year, quantity, price,
        rank() OVER (PARTITION BY product_id ORDER BY year) as year_no
    FROM sales) AS X
WHERE year_no=1
```

### 1075. Project Employees I

```mysql
SELECT project_id, ROUND(AVG(experience_years), 2) AS average_years
FROM project p
JOIN employee e
ON p.employee_id=e.employee_id
GROUP BY project_id
```

### 1076. Project Employees II

```mysql
/*SELECT project_id
FROM project
GROUP BY project_id
HAVING COUNT(employee_id)>=ALL (
    SELECT COUNT(employee_id) AS num
    FROM project
    GROUP BY project_id)*/
    
SELECT project_id
FROM project
GROUP BY project_id
HAVING COUNT(employee_id)=(
    SELECT COUNT(employee_id) AS cnt
    FROM project
    GROUP BY project_id
    ORDER BY CNT DESC
    limit 1
)
```

### 1077. Project Employees III

```mysql
-- MYSQL
SELECT p.project_id, e.employee_id
FROM project p
JOIN employee e
ON p.employee_id=e.employee_id
JOIN (
    SELECT project_id, MAX(experience_years) as years
    FROM project p
    JOIN employee e
    ON p.employee_id=e.employee_id
    GROUP BY project_id) x
ON p.project_id=x.project_id AND e.experience_years=x.years

-- SQL server
SELECT project_id, employee_id 
FROM (
    SELECT p.project_id, e.employee_id,
        RANK() OVER (PARTITION BY p.project_id ORDER BY e.experience_years DESC) AS rnk
    FROM project p
    JOIN employee e
    ON p.employee_id=e.employee_id) X
WHERE rnk=1
```

### 1082. Sales Analysis I

```mysql
SELECT seller_id
FROM sales
GROUP BY seller_id
HAVING SUM(price)=(
    SELECT SUM(price) AS price
    FROM sales
    GROUP BY seller_id
    ORDER BY price DESC
    LIMIT 1
)```

### 1083. Sales Analysis II

```mysql
# join-group by-case, cleaner query
SELECT s.buyer_id
FROM Sales AS s
JOIN Product AS p
ON s.product_id=p.product_id
GROUP BY s.buyer_id
HAVING SUM(CASE WHEN p.product_name = 'S8' THEN 1 ELSE 0 END) > 0
AND SUM(CASE WHEN p.product_name = 'iPhone' THEN 1 ELSE 0 END) = 0;

# typically join is faster than nested select 
/* SELECT DISTINCT(buyer_id)
FROM Sales
WHERE buyer_id IN
    (SELECT s.buyer_id
    FROM sales s JOIN product p
    ON s.product_id=p.product_id
    WHERE p.product_name='S8'
    )
AND buyer_id NOT IN
    (SELECT s.buyer_id
    FROM sales s JOIN product p
    ON s.product_id=p.product_id
    WHERE p.product_name='iphone'
    )*/

# nested select
/*SELECT DISTINCT buyer_id
FROM sales
WHERE buyer_id IN (
    SELECT buyer_id
    FROM sales
    WHERE product_id=(
        SELECT product_id
        FROM product
        WHERE product_name='S8'
    )
)
AND buyer_id NOT IN (
    SELECT buyer_id
    FROM sales
    WHERE product_id=(
        SELECT product_id
        FROM product
        WHERE product_name='iPhone'
    )
)*/
```

### 1084. Sales Analysis III

```mysql
# USE JOIN & GROUP BY
/* SELECT p.product_id, product_name
FROM product p
JOIN sales s
ON p.product_id=s.product_id
GROUP BY product_id
HAVING MIN(sale_date)>="2019-01-01"
    AND MAX(sale_date)<="2019-03-31"*/

# NO JOIN and GROUP BY, use NOT IN
SELECT p.product_id, product_name
FROM product p
WHERE p.product_id NOT IN (
    SELECT product_id
    FROM sales
    WHERE sale_date<"2019-01-01" OR sale_date>"2019-03-31"
)
```

### 1097. Game Play Analysis V

```mysql
SELECT install_dt, COUNT(DISTINCT player_id) AS installs,
    ROUND(SUM(relogin)/COUNT(DISTINCT player_id), 2) AS Day1_retention
FROM (
    SELECT a.player_id, install_dt,
        CASE WHEN a.install_dt+1 = a2.event_date THEN 1
        ELSE 0 END AS relogin
    FROM (
        SELECT player_id, MIN(event_date) AS install_dt
        FROM activity
        GROUP BY player_id) a
    JOIN activity a2
    ON a.player_id=a2.player_id) b
GROUP BY install_dt
```

### 1098. Unpopular Books

```mysql
/* SELECT a.book_id, a.name
FROM (
    SELECT book_id, name
    FROM books
    WHERE available_from<="2019-05-23"
) a
LEFT JOIN (
    SELECT book_id, sum(quantity) AS qty
    FROM orders
    WHERE dispatch_date>"2018-06-23"
    GROUP BY book_id) b
ON a.book_id=b.book_id
WHERE IFNULL(b.qty, 0)<10 */
#distinct

SELECT  b.book_id, b.name
FROM books b
LEFT JOIN orders o
ON o.book_id = b.book_id and o.dispatch_date >= '2018-06-23'
WHERE b.book_id not in (
    SELECT DISTINCT book_id FROM books WHERE available_from > '2019-05-23'
)
GROUP BY b.book_id
HAVING SUM(o.quantity) IS NULL OR SUM(o.quantity) < 10
```

### 1107. New Users Daily Count

```mysql
SELECT login_date, COUNT(user_id) AS user_count
FROM (
    SELECT user_id, MIN(activity_date) AS login_date
    FROM traffic
    WHERE activity='login'
    GROUP BY user_id) AS x
GROUP BY login_date
HAVING login_date >= DATE_ADD('2019-06-30', INTERVAL -90 DAY)
```